# Preambule

In [ ]:
# !pip install git+https://github.com/fra31/auto-attack

In [ ]:
# from autoattack import AutoAttack 
from tqdm import tqdm
import math
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
# https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
# Define relevant variables for the ML task
batch_size = 128
num_classes = 10
learning_rate = 0.001
num_epochs = 10
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'torch'

In [ ]:
device

In [ ]:
import random
def set_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed) 

# MC dropout & Ensemble

In [ ]:
#@title MC LeNet5
class Dropout2dLayer(nn.Dropout2d):
  def __init__(self, droprate=0.5, inplace=False):
    super().__init__(p=droprate, inplace=inplace)

  def forward(self, input):
    return F.dropout2d(input, self.p, training=True, inplace=self.inplace)

class MCLeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # num_classes = 10
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            Dropout2dLayer(0.2),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            Dropout2dLayer(0.2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(400, 120)
        # self.dropout2dlayer1 = Dropout2dLayer(0.5)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        # self.dropout2dlayer2 = Dropout2dLayer(0.5)
        self.relu1 = nn.ReLU()
        self.fc_head = nn.Linear(84, num_classes)

    def embedding(self, x):
        out = self.layer1(x)
        out = self.layer2(out)    
        
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        # out = self.dropout2dlayer1(out)
        out = F.dropout(x, 0.5, training=True, inplace=False)
        out = self.relu(out)

        out = self.fc1(out)
        # out = self.dropout2dlayer2(out)
        out = F.dropout(input, 0.5, training=True, inplace=False)
        out = self.relu1(out)
        return out

    def forward(self, x):
        self.e1 = self.embedding(x)
        out = self.fc_head(self.e1)
        return out
      

In [ ]:
#@title Ensemble/initialization
def init_weights(init_function, **kwargs):
  def _init_weights(module):
    if isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight, **kwargs)
        if module.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(module.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(module.bias, -bound, bound)

    if isinstance(module, nn.Conv2d):
        init_function(module.weight, **kwargs)
        if module.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(module.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                nn.init.uniform_(module.bias, -bound, bound)
  return _init_weights

In [ ]:
# a=math.sqrt(5)
init_list = [nn.init.kaiming_uniform_, nn.init.kaiming_normal_, nn.init.xavier_uniform_, nn.init.xavier_normal_, nn.init.uniform_]
initialization1 = init_weights(nn.init.kaiming_uniform_, a=math.sqrt(5))
# model.apply(initialization1)

In [ ]:
#@title sampler
generator = torch.Generator(device).manual_seed(4599)
sampler = torch.utils.data.RandomSampler(train_dataset, generator=generator)
dtl = torch.utils.data.DataLoader(train_dataset, batch_size=10, sampler=sampler)
for x, y in dtl:
  print(x.mean(), y.shape)
  break

# Train Eval 1

In [ ]:
#@title load mnist
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)

valid_size= 5000
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [len(train_dataset)-valid_size, valid_size])

# Make dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, pin_memory=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_dataset, pin_memory=True)


# train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
#                                            batch_size = batch_size,
#                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [ ]:
len(valid_loader)

In [ ]:
#@title cifar10_handler
class CIFAR10_Handler(Dataset):
    def __init__(self, X, Y, train=True):
        self.X = X
        self.Y = Y
        if train:
            self.transform = transforms.Compose(
                [
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(
                        (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
                    ),
                ]
            )
        else:
            self.transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(
                        (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
                    ),
                ]
            )

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        x = Image.fromarray(x)
        x = self.transform(x)
        return x, y, index

    def __len__(self):
        return len(self.X)

# Data.py
def get_CIFAR10(pool_size):
    data_train = datasets.CIFAR10('./data/CIFAR10', train=True, download=True)
    data_test = datasets.CIFAR10('./data/CIFAR10', train=False, download=True)
    dtrain =  CIFAR10_Handler(data_train.data[:pool_size], torch.LongTensor(data_train.targets)[:pool_size])
    dtest = CIFAR10_Handler(data_test.data[:pool_size], torch.LongTensor(data_test.targets)[:pool_size])
    return dtrain, dtest

In [ ]:
#@title cifar10
train_dataset, test_dataset = get_CIFAR10(45000)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [ ]:
#@title LeNet5
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # num_classes = 10
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc_head = nn.Linear(84, num_classes)

    def embedding(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        return out

    def forward(self, x):
        self.e1 = self.embedding(x)
        out = self.fc_head(self.e1)
        return out
      
    # def predict(self, x):
    #     with torch.no_grad():
    #       return self.forward(x)

In [ ]:
#@title resnet model
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
# src: https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def embedding(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        return out

    def forward(self, x):
        self.e1 = self.embedding(x)
        out = self.e1.view(self.e1.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
#@title Data Class
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset

class Data:
    def __init__(self, X_train, Y_train, X_test, Y_test, handler, n_adv_test):
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
        self.handler = handler

        self.n_pool = len(X_train)
        self.n_test = len(X_test)

        self.labeled_idxs = np.zeros(self.n_pool, dtype=bool)
        # To handle addition of adversarial dataset to labelled pool
        self.X_train_extra = torch.Tensor([])
        self.Y_train_extra = torch.Tensor([])
        # adv test data
        self.n_adv_test = n_adv_test
        if self.n_adv_test == self.n_test:
            self.adv_test_idxs = np.arange(self.n_test)
        else:
            self.adv_test_idxs = np.random.choice(
                np.arange(self.n_test), self.n_adv_test, replace=False)

    def initialize_labels(self, num):
        # generate initial labeled pool
        tmp_idxs = np.arange(self.n_pool)
        np.random.shuffle(tmp_idxs)
        # print(tmp_idxs[:5], tmp_idxs[-5:])
        self.labeled_idxs[tmp_idxs[:num]] = True

    def add_extra_data(self, pos_idxs, extra_data):
        # print('Y_train_extra', self.Y_train[pos_idxs])
        if len(self.X_train_extra) > 0:
            self.X_train_extra = torch.vstack([self.X_train_extra, extra_data])
            self.Y_train_extra = torch.hstack(
                [self.Y_train_extra, self.Y_train[pos_idxs]])
        else:
            self.X_train_extra = extra_data
            self.Y_train_extra = self.Y_train[pos_idxs]
        # assert len(self.X_train_extra) == len(self.Y_train_extra)
        # print('New Y_train_extra', self.Y_train_extra)

    def get_labeled_data(self):
        labeled_idxs = np.arange(self.n_pool)[self.labeled_idxs]
        if len(self.X_train_extra) > 0:
            # print('data.py:44',self.X_train[labeled_idxs].shape, self.X_train_extra.shape)
            if len(self.X_train_extra.shape) - len(self.X_train.shape) == -1:
                X_train_extra = self.X_train_extra.unsqueeze(1)
            elif len(self.X_train_extra.shape) - len(self.X_train.shape) == 1:
                X_train_extra = self.X_train_extra.squeeze(1)
            else:
                X_train_extra = self.X_train_extra
            # breakpoint()
            if torch.is_tensor(X_train_extra):
                self.X_train = torch.tensor(self.X_train)
                
            X = torch.vstack((self.X_train[labeled_idxs], X_train_extra))
            Y = torch.hstack([self.Y_train[labeled_idxs], self.Y_train_extra])
        else:
            X = self.X_train[labeled_idxs]
            Y = self.Y_train[labeled_idxs]
        return labeled_idxs, self.handler(X, Y)

    def n_labeled(self):
        return sum(self.labeled_idxs) + len(self.X_train_extra)

    def get_unlabeled_data(self, n_subset=None):
        unlabeled_idxs = np.arange(self.n_pool)[~self.labeled_idxs]
        np.random.shuffle(unlabeled_idxs)
        if n_subset:
            unlabeled_idxs = unlabeled_idxs[:n_subset]
        X = self.X_train[unlabeled_idxs]
        Y = self.Y_train[unlabeled_idxs]
        return unlabeled_idxs, self.handler(X, Y)

    def get_train_data(self):
        return self.labeled_idxs.copy(), self.handler(self.X_train, self.Y_train)

    def get_test_data(self):
        return self.handler(self.X_test, self.Y_test, train=False)

    def get_adv_test_data(self):
        return self.handler(self.X_test[self.adv_test_idxs], self.Y_test[self.adv_test_idxs], train=False)

    def cal_test_acc(self, preds):
        return 100.0 * (self.Y_test == preds).sum().item() / self.n_test

    def cal_adv_test_acc(self, preds):
        return 100.0 * (self.Y_test[self.adv_test_idxs] == preds).sum().item() / self.n_adv_test


def get_xMNIST(x_fn, handler, pool_size, n_adv_test, pref = ''):
    # transform_ = transforms.Compose([
    #                                               transforms.Resize((32,32)),
    #                                               transforms.ToTensor(),
    #                                               transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
    # transform_ = ToTensor()
    raw_train = x_fn(root='./data/'+pref+'MNIST', train=True, download=True, transform=transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]))
    raw_test = x_fn(root='./data/'+pref+'MNIST', train=False, download=True, transform=transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]))

    # raw_train = x_fn(root = './data',
    #                                           train = True,
    #                                           transform = transforms.Compose([
    #                                                   transforms.Resize((32,32)),
    #                                                   transforms.ToTensor(),
    #                                                   transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
    #                                           download = True)

    dtl = DataLoader(raw_train, batch_size=len(raw_train))
    for X, y in dtl:
        X_train = X
        Y_train = y

    dtl = DataLoader(raw_test, batch_size=len(raw_test))
    for X,y in dtl:
        X_test = X
        Y_test = y
    return Data(X_train[:pool_size], Y_train[:pool_size], X_test, Y_test, handler, n_adv_test)


def get_MNIST(handler, pool_size, n_adv_test):
    return get_xMNIST(datasets.MNIST, handler, pool_size, n_adv_test)

class MNIST_Handler(Dataset):
    def __init__(self, X, Y, train=True):
        self.X = X
        self.Y = Y
        # self.transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        # if not isinstance(x, np.ndarray):
        #     x = Image.fromarray(x.numpy(), mode='L')
        #     x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.X)

data = get_MNIST(MNIST_Handler, 45000, 300)

In [ ]:
# data.initialize_labels(60000)
# _,trd = data.get_labeled_data()
# train_loader = DataLoader(trd, shuffle=True, batch_size=batch_size)

In [ ]:
model = LeNet5().to(device)
# model = MCLeNet5().to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [ ]:
#@title resnet train

# Define relevant variables for the ML task
batch_size = 256
num_classes = 10
learning_rate = 0.1
num_epochs = 100
optparams = {'weight_decay': 0.0005, 'momentum': 0.9}
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ResNet18().to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, **optparams)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
#
#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [ ]:
print(model)

In [ ]:
# batch_size = 8
# train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
#                                            batch_size = batch_size,
#                                            shuffle = True)
# for i, (images, labels) in enumerate(train_loader):
#   print(images.shape, type(images), images.dtype)
#   break

In [ ]:
total_step = len(train_loader)
model.train()  # set train mode
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
model.eval()

In [ ]:
#@title Clean Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

# Temp Scaling

In [ ]:
class ModelWithTemperature(nn.Module):
    """
    A thin decorator, which wraps a model with temperature scaling
    model (nn.Module):
        A classification neural network
        NB: Output of the neural network should be the classification logits,
            NOT the softmax (or log softmax)!
    """
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = temperature = nn.Parameter(torch.ones(1).cuda())

    def forward(self, input):
        logits = self.model(input)
        return self.temperature_scale(logits)

    def temperature_scale(self, logits):
        """
        Perform temperature scaling on logits
        """
        # Expand temperature to match the size of logits
        temperature = self.temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))
        return logits / temperature

    def T_scaling(self, logits):
        return torch.div(logits, self.temperature)

    # This function probably should live outside of this class, but whatever
    def set_temperature(self, valid_loader):
        """
        Tune the tempearature of the model (using the validation set).
        We're going to set it to optimize NLL.
        valid_loader (DataLoader): validation set loader
        """
        self.to(device)
        nll_criterion = nn.CrossEntropyLoss().to(device)
        ece_criterion = _ECELoss().to(device)

        # First: collect all the logits and labels for the validation set
        logits_list = []
        labels_list = []
        temps = []
        losses = []

        for i, data in enumerate(tqdm(valid_loader, 0)):
            images, labels = data[0].to(device), data[1].to(device)

            self.model.eval()
            with torch.no_grad():
                logits_list.append(self.model(images))
                labels_list.append(labels)

        # Create tensors
        logits_list = torch.cat(logits_list).to(device)
        labels_list = torch.cat(labels_list).to(device)


        # Calculate NLL and ECE before temperature scaling
        before_temperature_nll = nll_criterion(logits_list, labels).item()
        before_temperature_ece = ece_criterion(logits_list, labels).item()
        print('Before temperature - NLL: %.3f, ECE: %.3f' % (before_temperature_nll, before_temperature_ece))

        # Next: optimize the temperature w.r.t. NLL
        # Removing strong_wolfe line search results in jump after 50 epochs
        optimizer = torch.optim.LBFGS([self.temperature], lr=0.001, max_iter=10000, line_search_fn='strong_wolfe')

    
        def _eval():
            loss = nll_criterion(self.T_scaling(logits_list), labels_list)
            loss.backward()
            temps.append(self.temperature.item())
            losses.append(loss)
            return loss


        optimizer.step(_eval)
        
        # Calculate NLL and ECE after temperature scaling
        after_temperature_nll = nll_criterion(self.temperature_scale(logits_list), labels).item()
        after_temperature_ece = ece_criterion(self.temperature_scale(logits_list), labels).item()
        print('Optimal temperature: %.3f' % self.temperature.item())
        print('After temperature - NLL: %.3f, ECE: %.3f' % (after_temperature_nll, after_temperature_ece))

        plt.subplot(121)
        plt.plot(list(range(len(temps))), temps)

        plt.subplot(122)
        xlosses = [x.detach().cpu() for x in losses]
        plt.plot(list(range(len(xlosses))), xlosses)
        plt.show()


        return self

class _ECELoss(nn.Module):
    """
    Calculates the Expected Calibration Error of a model.
    (This isn't necessary for temperature scaling, just a cool metric).

    The input to this loss is the logits of a model, NOT the softmax scores.

    This divides the confidence outputs into equally-sized interval bins.
    In each bin, we compute the confidence gap:

    bin_gap = | avg_confidence_in_bin - accuracy_in_bin |

    We then return a weighted average of the gaps, based on the number
    of samples in each bin

    See: Naeini, Mahdi Pakdaman, Gregory F. Cooper, and Milos Hauskrecht.
    "Obtaining Well Calibrated Probabilities Using Bayesian Binning." AAAI.
    2015.
    """
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(_ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [ ]:
#@title https://github.com/gpleiss/temperature_scaling
import torch
from torch import nn, optim
from torch.nn import functional as F


class ModelWithTemperature(nn.Module):
    """
    A thin decorator, which wraps a model with temperature scaling
    model (nn.Module):
        A classification neural network
        NB: Output of the neural network should be the classification logits,
            NOT the softmax (or log softmax)!
    """
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, input):
        logits = self.model(input)
        return self.temperature_scale(logits)

    def temperature_scale(self, logits):
        """
        Perform temperature scaling on logits
        """
        # Expand temperature to match the size of logits
        temperature = self.temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))
        return logits / temperature

    # This function probably should live outside of this class, but whatever
    def set_temperature(self, valid_loader):
        """
        Tune the tempearature of the model (using the validation set).
        We're going to set it to optimize NLL.
        valid_loader (DataLoader): validation set loader
        """
        self.to(device)
        nll_criterion = nn.CrossEntropyLoss().to(device)
        ece_criterion = _ECELoss().to(device)

        # First: collect all the logits and labels for the validation set
        logits_list = []
        labels_list = []
        with torch.no_grad():
            for input, label in valid_loader:
                input = input.to(device)
                logits = self.model(input)
                logits_list.append(logits)
                labels_list.append(label)
            logits = torch.cat(logits_list).to(device)
            labels = torch.cat(labels_list).to(device)

        # Calculate NLL and ECE before temperature scaling
        before_temperature_nll = nll_criterion(logits, labels).item()
        before_temperature_ece = ece_criterion(logits, labels).item()
        print('Before temperature - NLL: %.3f, ECE: %.3f' % (before_temperature_nll, before_temperature_ece))

        # Next: optimize the temperature w.r.t. NLL
        optimizer = optim.LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval():
            optimizer.zero_grad()
            loss = nll_criterion(self.temperature_scale(logits), labels)
            loss.backward()
            return loss
        optimizer.step(eval)

        # Calculate NLL and ECE after temperature scaling
        after_temperature_nll = nll_criterion(self.temperature_scale(logits), labels).item()
        after_temperature_ece = ece_criterion(self.temperature_scale(logits), labels).item()
        print('Optimal temperature: %.3f' % self.temperature.item())
        print('After temperature - NLL: %.3f, ECE: %.3f' % (after_temperature_nll, after_temperature_ece))

        return self


class _ECELoss(nn.Module):
    """
    Calculates the Expected Calibration Error of a model.
    (This isn't necessary for temperature scaling, just a cool metric).

    The input to this loss is the logits of a model, NOT the softmax scores.

    This divides the confidence outputs into equally-sized interval bins.
    In each bin, we compute the confidence gap:

    bin_gap = | avg_confidence_in_bin - accuracy_in_bin |

    We then return a weighted average of the gaps, based on the number
    of samples in each bin

    See: Naeini, Mahdi Pakdaman, Gregory F. Cooper, and Milos Hauskrecht.
    "Obtaining Well Calibrated Probabilities Using Bayesian Binning." AAAI.
    2015.
    """
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(_ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [ ]:
# orig_model = ... # create an uncalibrated model somehow
# valid_loader = ... # Create a DataLoader from the SAME VALIDATION SET used to train orig_model
scaled_model = ModelWithTemperature(model)
scaled_model.set_temperature(valid_loader)

In [ ]:
#@title Clean Test scaled model
# In test phase, we don't need to compute gradients (for memory efficiency)
scaled_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = scaled_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

In [ ]:
#@title temp scale
nll_criterion = nn.CrossEntropyLoss().to(device)
ece_criterion = _ECELoss().to(device)
# First: collect all the logits and labels for the validation set
logits_list = []
labels_list = []
with torch.no_grad():
    for input, label in test_loader:
        input = input.to(device)
        logits = model(input)
        logits_list.append(logits)
        labels_list.append(label)
    logits = torch.cat(logits_list).to(device)
    labels = torch.cat(labels_list).to(device)

# Calculate NLL and ECE before temperature scaling
before_temperature_nll = nll_criterion(logits, labels).item()
before_temperature_ece = ece_criterion(logits, labels).item()
print('Before temperature - NLL: %.3f, ECE: %.3f' % (before_temperature_nll, before_temperature_ece))

In [ ]:
#@title temp scale
nll_criterion = nn.CrossEntropyLoss().to(device)
ece_criterion = _ECELoss().to(device)
# First: collect all the logits and labels for the validation set
logits_list = []
labels_list = []
with torch.no_grad():
    for input, label in test_loader:
        input = input.to(device)
        logits = scaled_model(input)
        logits_list.append(logits)
        labels_list.append(label)
    logits = torch.cat(logits_list).to(device)
    labels = torch.cat(labels_list).to(device)

after_temperature_nll = nll_criterion(scaled_model.temperature_scale(logits), labels).item()
after_temperature_ece = ece_criterion(scaled_model.temperature_scale(logits), labels).item()
print('Optimal temperature: %.3f' % scaled_model.temperature.item())
print('After temperature - NLL: %.3f, ECE: %.3f' % (after_temperature_nll, after_temperature_ece))

In [ ]:
#@title https://github.com/google/uncertainty-baselines/blob/main/baselines/mnist/utils.py
# coding=utf-8
# Copyright 2022 The Uncertainty Baselines Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Utilities for (Fashion) MNIST."""

import numpy as np
import scipy


def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])


def brier_score(y, p):
  """Compute the Brier score.

  Brier Score: see
  https://www.stat.washington.edu/raftery/Research/PDF/Gneiting2007jasa.pdf,
  page 363, Example 1

  Args:
    y: one-hot encoding of the true classes, size (?, num_classes)
    p: numpy array, size (?, num_classes)
       containing the output predicted probabilities
  Returns:
    bs: Brier score.
  """
  return np.mean(np.power(p - y, 2))


def calibration(y, p_mean, num_bins=10):
  """Compute the calibration.

  References:
  https://arxiv.org/abs/1706.04599
  https://arxiv.org/abs/1807.00263

  Args:
    y: one-hot encoding of the true classes, size (?, num_classes)
    p_mean: numpy array, size (?, num_classes)
           containing the mean output predicted probabilities
    num_bins: number of bins

  Returns:
    ece: Expected Calibration Error
    mce: Maximum Calibration Error
  """
  # Compute for every test sample x, the predicted class.
  class_pred = np.argmax(p_mean, axis=1)
  # and the confidence (probability) associated with it.
  conf = np.max(p_mean, axis=1)
  # Convert y from one-hot encoding to the number of the class
  y = np.argmax(y, axis=1)
  # Storage
  acc_tab = np.zeros(num_bins)  # empirical (true) confidence
  mean_conf = np.zeros(num_bins)  # predicted confidence
  nb_items_bin = np.zeros(num_bins)  # number of items in the bins
  tau_tab = np.linspace(0, 1, num_bins+1)  # confidence bins
  for i in np.arange(num_bins):  # iterate over the bins
    # select the items where the predicted max probability falls in the bin
    # [tau_tab[i], tau_tab[i + 1)]
    sec = (tau_tab[i + 1] > conf) & (conf >= tau_tab[i])
    nb_items_bin[i] = np.sum(sec)  # Number of items in the bin
    # select the predicted classes, and the true classes
    class_pred_sec, y_sec = class_pred[sec], y[sec]
    # average of the predicted max probabilities
    mean_conf[i] = np.mean(conf[sec]) if nb_items_bin[i] > 0 else np.nan
    # compute the empirical confidence
    acc_tab[i] = np.mean(
        class_pred_sec == y_sec) if nb_items_bin[i] > 0 else np.nan

  # Cleaning
  mean_conf = mean_conf[nb_items_bin > 0]
  acc_tab = acc_tab[nb_items_bin > 0]
  nb_items_bin = nb_items_bin[nb_items_bin > 0]

  # Expected Calibration Error
  ece = np.average(
      np.absolute(mean_conf - acc_tab),
      weights=nb_items_bin.astype(float) / np.sum(nb_items_bin))
  # Maximum Calibration Error
  mce = np.max(np.absolute(mean_conf - acc_tab))
  return ece, mce


def ensemble_metrics(x,
                     y,
                     model,
                     log_likelihood_fn,
                     n_samples=1,
                     weight_files=None):
  """Evaluate metrics of an ensemble.

  Args:
    x: numpy array of inputs
    y: numpy array of labels
    model: tf.keras.Model.
    log_likelihood_fn: keras function of log likelihood. For classification
      tasks, log_likelihood_fn(...)[1] should return the logits
    n_samples: number of Monte Carlo samples to draw per ensemble member (each
      weight file).
    weight_files: to draw samples from multiple weight sets, specify a list of
      weight files to load. These files must have been generated through
      keras's model.save_weights(...).

  Returns:
    metrics_dict: dictionary containing the metrics
  """
  if weight_files is None:
    ensemble_logprobs = [log_likelihood_fn([x, y])[0] for _ in range(n_samples)]
    metric_values = [model.evaluate(x, y, verbose=0)
                     for _ in range(n_samples)]
    ensemble_logits = [log_likelihood_fn([x, y])[1] for _ in range(n_samples)]
  else:
    ensemble_logprobs = []
    metric_values = []
    ensemble_logits = []
    for filename in weight_files:
      model.load_weights(filename)
      ensemble_logprobs.extend([log_likelihood_fn([x, y])[0]
                                for _ in range(n_samples)])
      ensemble_logits.extend([log_likelihood_fn([x, y])[1]
                              for _ in range(n_samples)])
      metric_values.extend([model.evaluate(x, y, verbose=0)
                            for _ in range(n_samples)])

  metric_values = np.mean(np.array(metric_values), axis=0)
  results = {}
  for m, name in zip(metric_values, model.metrics_names):
    results[name] = m

  ensemble_logprobs = np.array(ensemble_logprobs)
  probabilistic_log_likelihood = np.mean(
      scipy.special.logsumexp(
          np.sum(ensemble_logprobs, axis=2)
          if len(ensemble_logprobs.shape) > 2 else ensemble_logprobs,
          b=1. / ensemble_logprobs.shape[0],
          axis=0),
      axis=0)
  results['probabilistic_log_likelihood'] = probabilistic_log_likelihood

  ensemble_logits = np.array(ensemble_logits)
  probs = np.mean(scipy.special.softmax(ensemble_logits, axis=2), axis=0)
  class_pred = np.argmax(probs, axis=1)
  probabilistic_accuracy = np.mean(np.equal(y, class_pred))
  results['probabilistic_accuracy'] = probabilistic_accuracy
  results['ece'], results['mce'] = calibration(
      one_hot(y, probs.shape[1]), probs)
  results['brier_score'] = brier_score(one_hot(y, probs.shape[1]), probs)
  return results

# Adv Test

In [ ]:
# epsilon = 0.3
# adversary = AutoAttack(model, norm='Linf', eps=epsilon, version='standard', verbose=False)

In [ ]:
#@title AutoAttack Test the model (commented out)
# # In test phase, we don't need to compute gradients (for memory efficiency)
# test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
#                                            batch_size = 1,
#                                            shuffle = False)  
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in tqdm(test_loader):
#         images = images.to(device)
#         labels = labels.to(device)
#         x_adv = adversary.run_standard_evaluation(images, labels)
#         outputs = model(x_adv)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
%pip install cleverhans

In [ ]:
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent as pgd
from cleverhans.torch.attacks.carlini_wagner_l2 import carlini_wagner_l2 

In [ ]:
#@title adv_acc 300
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)
pgd_params = {'eps': 0.1, 'eps_iter': 0.01, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True} #, 'rand_minmax': 0.1}
cw_params = {' n_classes':10, 'targeted': False}
correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(1000))
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = 1,
                                           shuffle = False)  
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = pgd(model, images, **pgd_params)
    outputs = model(images)
    xoutputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('Accuracy of the network on {} test images: {} %'.format(total, 100 * correct / total))
print('Accuracy of the network on {} test images: {} %'.format(total, 100 * xcorrect / total))

In [ ]:
#@title adv_acc 300
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)
pgd_params = {'eps': 0.1, 'eps_iter': 0.01, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True} #, 'rand_minmax': 0.1}
cw_params = {'n_classes':10, 'targeted': False}
correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(1000))
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = 1,
                                           shuffle = False)  
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = carlini_wagner_l2(model, images, **cw_params)
    outputs = model(images)
    xoutputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('Accuracy of the network on {} test images: {} %'.format(total, 100 * correct / total))
print('Accuracy of the network on {} test images: {} %'.format(total, 100 * xcorrect / total))

In [ ]:
# #@title full 10k adv test
# # Test the model
# # In test phase, we don't need to compute gradients (for memory efficiency)
# set_seeds(42)
# pgd_params = {'eps': 0.3, 'eps_iter': 0.1, 'nb_iter': 50, 'norm': np.inf, 'targeted': False, 'rand_init': True}

# correct = 0
# total = 0
# distances = []
# test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
#                                            batch_size = 1,
#                                            shuffle = False)  
# for images, labels in tqdm(test_loader):
#     images = images.to(device)
#     labels = labels.to(device)
#     x_adv = pgd(model, images,  **pgd_params)
#     # assert(x_adv.shape==images.shape)
#     outputs = model(x_adv)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
#@title duplicate above with different args (eps_iter, rand_minmax)
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)
pgd_params = {'eps': 0.3, 'eps_iter': 0.1, 'nb_iter': 50, 'norm': np.inf, 'targeted': False, 'rand_init': True, 'rand_minmax': 0.1}

correct = 0
total = 0
distances = []
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = 1,
                                           shuffle = False)  
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = pgd(model, images,  **pgd_params)
    assert(x_adv.shape==images.shape)
    outputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# show(images, predicted, labels, 'clean')
# show(x_adv, xpredicted, labels, 'clean')

# Adversarial Train Test 

In [ ]:
robust_model = LeNet5().to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [ ]:
#@title adversarial training
pgd_params = {'eps': 0.1, 'eps_iter': 0.01, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True} #, 'rand_minmax': 0.1}
total_step = len(train_loader)
robust_model.train()
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        #attack
        x_adv = pgd(robust_model, images,  **pgd_params)
        delta = (images - x_adv).detach()
        #Forward pass
        outputs = model(images + delta)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
robust_model.eval()

In [ ]:
#@title Clean Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = robust_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

In [ ]:
#@title adv_acc 300
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)
pgd_params = {'eps': 0.1, 'eps_iter': 0.01, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True} #, 'rand_minmax': 0.1}
correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(1000))
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = 1,
                                          shuffle = False)  

for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = pgd(model, images, **pgd_params)
    xoutputs = robust_model(x_adv.detach())
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('AAccuracy of the network on {} test images: {} %'.format(total, 100 * xcorrect / total))

# Others attacks

In [ ]:
#@title deefool
def compute_norm(x, norm):
    with torch.no_grad():
        if norm == np.inf:
            return torch.linalg.norm(torch.ravel(x.cpu()), ord=np.inf).numpy()
        elif norm == 2:
            return torch.linalg.norm(x.cpu()).numpy()
        else:
            raise NotImplementedError

def deepfool_attack(model, x, max_iter, **args):
    """DeepFool attack"""
    nx = x.clone()
    nx.requires_grad_()
    eta = torch.zeros(nx.shape).cuda() if torch.cuda.is_available() else torch.zeros(nx.shape)

    out = model(nx+eta)
    n_class = out.shape[1]
    initial_label = out.max(1)[1]
    pred_nx = out.max(1)[1]

    i_iter = 0
    cumul_dis_2 = 0.
    cumul_dis_inf = 0.
    while pred_nx == initial_label and i_iter < max_iter:
        out[0, pred_nx].backward(retain_graph=True)
        grad_np = nx.grad.data.clone()
        value_l = np.inf
        w_l = None
        for i in range(n_class):
            if i == initial_label:
                continue

            nx.grad.data.zero_()
            out[0, i].backward(retain_graph=True)
            grad_i = nx.grad.data.clone()

            wi = grad_i - grad_np
            fi = out[0, i] - out[0, initial_label]
            value_i = np.abs(fi.item()) / torch.norm(wi.flatten())
            # breakpoint()
            if value_i < value_l:
                value_l = value_i
                w_l = wi

        ri = value_l/torch.norm(w_l.flatten()) * w_l
        #
        cumul_dis_inf += compute_norm(ri, norm=np.inf)
        cumul_dis_2 += compute_norm(ri, norm=2)
        #
        eta += ri.clone()
        nx.grad.data.zero_()
        out = model(nx+eta)
        pred_nx = out.max(1)[1]
        i_iter += 1

    cumul_dis = {'2': cumul_dis_2, 'inf': cumul_dis_inf}
    return x+ri, i_iter, cumul_dis


def test_deepfool_attack(model, x, y=None, **args):
    """DeepFool attack"""
    return deepfool_attack(model, x, max_iter=args['nb_iter'])[0]

In [ ]:
#@title test deepfool
#@title adv_acc 300
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)

correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(1000))
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = 1,
                                           shuffle = False)
args = {'nb_iter': 100}  
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = test_deepfool_attack(model, images, **args)
    outputs = model(images)
    xoutputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * xcorrect / total))

In [ ]:
#@title test autoattack
#@title adv_acc 300
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)

correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(1000))
bs = 100
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = bs,
                                           shuffle = False)
epsilon = 0.3
adversary = AutoAttack(model, norm='Linf', eps=epsilon, version='standard', verbose=False)

for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = adversary.run_standard_evaluation(images, labels, bs=bs)
    outputs = model(images)
    xoutputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
print('AAccuracy of the network on the 10000 test images: {} %'.format(100 * xcorrect / total))

In [ ]:
#@title pgd_adaptive
"""The Projected Gradient Descent attack.
Original from cleverhans: https://github.com/cleverhans-lab/cleverhans/blob/master/cleverhans/torch/attacks/projected_gradient_descent.py
Edited by: Jonas Ngnawe
"""

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.utils import clip_eta


def compute_norm(x, norm):
    if norm == np.inf:
        return torch.linalg.norm(torch.ravel(x.cpu()), ord=np.inf)
    elif norm == 2:
        return torch.linalg.norm(x.cpu())
    else:
        raise NotImplementedError

def xpgd(
    model_fn,
    x,
    eps,
    eps_iter,
    nb_iter,
    norm,
    clip_min=None,
    clip_max=None,
    y=None,
    targeted=False,
    rand_init=True,
    rand_minmax=None,
    sanity_checks=True,
):
    """
    This class implements either the Basic Iterative Method
    (Kurakin et al. 2016) when rand_init is set to False. or the
    Madry et al. (2017) method if rand_init is set to True.
    Paper link (Kurakin et al. 2016): https://arxiv.org/pdf/1607.02533.pdf
    Paper link (Madry et al. 2017): https://arxiv.org/pdf/1706.06083.pdf
    :param model_fn: a callable that takes an input tensor and returns the model logits.
    :param x: input tensor.
    :param eps: epsilon (input variation parameter); see https://arxiv.org/abs/1412.6572.
    :param eps_iter: step size for each attack iteration
    :param nb_iter: Number of attack iterations.
    :param norm: Order of the norm (mimics NumPy). Possible values: np.inf, 1 or 2.
    :param clip_min: (optional) float. Minimum float value for adversarial example components.
    :param clip_max: (optional) float. Maximum float value for adversarial example components.
    :param y: (optional) Tensor with true labels. If targeted is true, then provide the
              target label. Otherwise, only provide this parameter if you'd like to use true
              labels when crafting adversarial samples. Otherwise, model predictions are used
              as labels to avoid the "label leaking" effect (explained in this paper:
              https://arxiv.org/abs/1611.01236). Default is None.
    :param targeted: (optional) bool. Is the attack targeted or untargeted?
              Untargeted, the default, will try to make the label incorrect.
              Targeted will instead try to move in the direction of being more like y.
    :param rand_init: (optional) bool. Whether to start the attack from a randomly perturbed x.
    :param rand_minmax: (optional) bool. Support of the continuous uniform distribution from
              which the random perturbation on x was drawn. Effective only when rand_init is
              True. Default equals to eps.
    :param sanity_checks: bool, if True, include asserts (Turn them off to use less runtime /
              memory or for unit tests that intentionally pass strange input)
    :return: a tensor for the adversarial example
    """
    if norm == 1:
        raise NotImplementedError(
            "It's not clear that FGM is a good inner loop"
            " step for PGD when norm=1, because norm=1 FGM "
            " changes only one pixel at a time. We need "
            " to rigorously test a strong norm=1 PGD "
            "before enabling this feature."
        )
    if norm not in [np.inf, 2]:
        raise ValueError("Norm order must be either np.inf or 2.")
    if eps < 0:
        raise ValueError(
            "eps must be greater than or equal to 0, got {} instead".format(
                eps)
        )
    if eps == 0:
        return x
    if eps_iter < 0:
        raise ValueError(
            "eps_iter must be greater than or equal to 0, got {} instead".format(
                eps_iter
            )
        )
    if eps_iter == 0:
        return x

    assert eps_iter <= eps, (eps_iter, eps)
    if clip_min is not None and clip_max is not None:
        if clip_min > clip_max:
            raise ValueError(
                "clip_min must be less than or equal to clip_max, got clip_min={} and clip_max={}".format(
                    clip_min, clip_max
                )
            )

    asserts = []

    # If a data range was specified, check that the input was in that range
    if clip_min is not None:
        assert_ge = torch.all(
            torch.ge(x, torch.tensor(clip_min, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_ge)

    if clip_max is not None:
        assert_le = torch.all(
            torch.le(x, torch.tensor(clip_max, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_le)

    # Initialize loop variables
    if rand_init:
        if rand_minmax is None:
            rand_minmax = eps
        eta = torch.zeros_like(x).uniform_(-rand_minmax, rand_minmax)
    else:
        eta = torch.zeros_like(x)

    # Clip eta
    eta = clip_eta(eta, norm, eps)
    adv_x = x + eta
    if clip_min is not None or clip_max is not None:
        adv_x = torch.clamp(adv_x, clip_min, clip_max)

    if y is None:
        # Using model predictions as ground truth to avoid label leaking
        _, y = torch.max(model_fn(x), 1)

    i = 0
    cumul_dis = 0.
    # Jonas: nb_iter will now act as max number of iterations (supposed very high)
    while i < nb_iter and torch.max(model_fn(adv_x), 1)[1] == y:
        tmp_x = adv_x.clone()
        adv_x = fast_gradient_method(
            model_fn,
            adv_x,
            eps_iter,
            norm,
            clip_min=clip_min,
            clip_max=clip_max,
            y=y,
            targeted=targeted,
        )
        #
        delta = adv_x - tmp_x
        cumul_dis += compute_norm(delta, norm)
        # Clipping perturbation eta to norm norm ball
        eta = adv_x - x
        eta = clip_eta(eta, norm, eps)
        adv_x = x + eta

        # Redo the clipping.
        # FGM already did it, but subtracting and re-adding eta can add some
        # small numerical error.
        if clip_min is not None or clip_max is not None:
            adv_x = torch.clamp(adv_x, clip_min, clip_max)
        i += 1

    asserts.append(eps_iter <= eps)
    if norm == np.inf and clip_min is not None:
        # TODO necessary to cast clip_min and clip_max to x.dtype?
        asserts.append(eps + clip_min <= clip_max)

    if sanity_checks:
        assert np.all(asserts)

    return adv_x, i, cumul_dis


In [ ]:
#@title eval dis
# from re import X
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
set_seeds(42)
pgd_params = {'eps': 0.3, 'eps_iter': 0.1, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True}

correct = 0
xcorrect = 0
total = 0
distances = []
xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(300))
test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                           batch_size = 1,
                                           shuffle = False)  
dis_list = []
iter_list = []
k=0
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv, i, cumul_dis= xpgd(model, images, **pgd_params)
    if torch.is_tensor(cumul_dis):
          cumul_dis = cumul_dis.detach().numpy()
    dis_list.append(cumul_dis)
    iter_list.append(i)
    
    # if i == 0:
    #   k = k+1
    # if k == 5:
    #   break
    outputs = model(images)
    xoutputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    _, xpredicted = torch.max(xoutputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    xcorrect += (xpredicted == labels).sum().item()
    # print()
    # show(images, predicted, labels, 'clean')
    # show(x_adv, xpredicted, labels, 'adv')
    # print('---')
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * xcorrect / total))
print(np.mean(dis_list))

In [ ]:
# show(images, predicted, labels, 'clean')
# show(x_adv, xpredicted, labels, 'clean')

In [ ]:
# eps=0.3
# rand_minmax = eps
# eta = torch.zeros_like(images).uniform_(-rand_minmax, rand_minmax)
# eta = clip_eta(eta, np.inf, eps)
# adv_x = images + eta
# _, xxpredicted = torch.max(xoutputs.data, 1)
# show(adv_x, xxpredicted, predicted, 'adv_x')

In [ ]:
iter_list

In [ ]:
#@title study function
def adv_acc_var(eps_iter, nb_iter=20):
  set_seeds(42)
  pgd_params = {'eps': 0.3, 'eps_iter': eps_iter, 'nb_iter': nb_iter, 'norm': np.inf, 'targeted': False, 'rand_init': True}

  correct = 0
  xcorrect = 0
  total = 0
  distances = []
  xtest_dataset = torch.utils.data.Subset(test_dataset, np.arange(100))
  test_loader = torch.utils.data.DataLoader(dataset = xtest_dataset,
                                            batch_size = 1,
                                            shuffle = False)  
  for images, labels in tqdm(test_loader):
      images = images.to(device)
      labels = labels.to(device)
      x_adv = pgd(model, images, y = labels, **pgd_params)
      outputs = model(images)
      xoutputs = model(x_adv)
      _, predicted = torch.max(outputs.data, 1)
      _, xpredicted = torch.max(xoutputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      xcorrect += (xpredicted == labels).sum().item()
      # print()
      # show(images, predicted, labels, 'clean')
      # show(x_adv, xpredicted, labels, 'adv')
      # print('---')
  acc = 100 * correct / total
  adv_acc = 100 * xcorrect / total 
  # print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
  # print('Accuracy of the network on the 10000 test images: {} %'.format(100 * xcorrect / total))
  return acc, adv_acc

In [ ]:
eps_iter_list = [0.001, 0.01, 0.05, 0.1]
nb_iter_list = [20]
accL=[]
adv_accL = []
# k = 0
for eps_iter in eps_iter_list:
  for nb_iter in nb_iter_list:
    acc, adv_acc = adv_acc_var(eps_iter, nb_iter)
    accL.append(acc)
    adv_accL.append(adv_acc)

plt.plot(accL)
plt.figure()
plt.plot(adv_accL)

In [ ]:
adv_accL, accL

In [ ]:
list(zip(eps_iter_list, nb_iter_list))

In [ ]:
eps_iter_list = [0.001, 0.01, 0.05, 0.1]
nb_iter_list = [500, 200, 50, 20]
accL=[]
adv_accL = []
for eps_iter, nb_iter in list(zip(eps_iter_list, nb_iter_list)):
    acc, adv_acc = adv_acc_var(eps_iter, nb_iter)
    accL.append(acc)
    adv_accL.append(adv_acc)

plt.plot(eps_iter_list, accL)
plt.figure()
plt.plot(eps_iter_list, adv_accL)

In [ ]:
import matplotlib.pyplot as plt
def show(im, predicted, label, txt):
# im1 = images[0][0].cpu().numpy()
  plt.figure()
  im = im[0][0].detach().cpu().numpy()
  plt.imshow(im)
  # print(predicted.cpu().numpy(), label.cpu().numpy(), (label==predicted).cpu().numpy())
  title = txt+' {} {} {}'.format(predicted.cpu().numpy(), label.cpu().numpy(), (label==predicted).cpu().numpy())
  plt.title(title)

In [ ]:
%pip install foolbox

In [ ]:
import foolbox as fb

fmodel = fb.PyTorchModel(model, bounds=(-5, 5))
attack = fb.attacks.LinfPGD()
epsilons = [0.3]
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
pgd_params = {'eps': 0.3, 'eps_iter': 0.05, 'nb_iter': 50, 'norm': np.inf, 'targeted': False, 'rand_init': True}

correct = 0
total = 0
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    total += labels.size(0)
    correct += success.sum().item()

print('FAccuracy of the network on the 10000 test images: {} %'.format(100 * (1 - correct) / total))

In [ ]:
# show(advs[0], predicted, labels, 'x')

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
pgd_params = {'eps': 0.3, 'eps_iter': 0.05, 'nb_iter': 50, 'norm': np.inf, 'targeted': False, 'rand_init': True}

correct = 0
total = 0
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = pgd(model, images, y=labels, **pgd_params)
    outputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
pgd_params = {'eps': 0.3, 'eps_iter': 0.05, 'nb_iter': 100, 'norm': np.inf, 'targeted': False, 'rand_init': True}

correct = 0
total = 0
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    x_adv = pgd(model, images, y=labels, **pgd_params)
    outputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
%%time
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
pgd_params = {'eps': 0.3, 'eps_iter': 0.1, 'nb_iter': 20, 'norm': np.inf, 'targeted': False, 'rand_init': True}

correct = 0
total = 0

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = 1,
                                           shuffle = False)
for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    x_adv = pgd(model, images, y=labels, **pgd_params)
    outputs = model(x_adv)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Distances eval

# Visualization

In [ ]:
test_dataset.data.shape

In [ ]:
# We want to get TSNE embedding with 2 dimensions
N=100
idxs = np.arange(len(test_dataset))
random.shuffle(idxs)
idxs = idxs[:N]
n_components = 2
tsne = TSNE(n_components,  learning_rate='auto', init='random', perplexity=30)
tsne_result = tsne.fit_transform(test_dataset.data[idxs].reshape(N,-1)/255.)
tsne_result.shape
# (1000, 2)
# Two dimensions for each of our images
 
# Plot the result of our TSNE with the label color coded
# A lot of the stuff here is about making the plot look pretty and not TSNE
tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': y})
fig, ax = plt.subplots(1)
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax,s=120)
lim = (tsne_result.min()-5, tsne_result.max()+5)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
# import numpy as np
# from sklearn.manifold import TSNE
X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(X)
X_embedded.shape, X.shape

# Metrics/ART

In [ ]:
%pip install adversarial-robustness-toolbox

In [ ]:
import art
from art.estimators.classification.pytorch import PyTorchClassifier
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance

In [ ]:
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance

In [ ]:
import copy

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = 1,
                                           shuffle = False)  

for images, labels in test_loader:
  min_pixel_value, max_pixel_value = images.min(), images.max()
  break
# Get the classifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=None,
    optimizer=None,
    input_shape=(1, 32, 32),
    nb_classes=10,
)
# classifier = copy.deepcopy(model)
# Get the classifier
# krc.fit(x_train, y_train, batch_size=batch_size, nb_epochs=1, verbose=0)
R_Linf = 0.3
R_L2 = 2

# Test targeted clever
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = 1,
                                           shuffle = False)
for images, labels in test_loader:
  # images, labels = images.to(device), labels.to(device)
  predictions = classifier.predict(images.cpu())
  print((predictions.shape, np.argmax(predictions, axis=1) == labels.numpy()))
  accuracy = np.sum(np.argmax(predictions, axis=1) == labels.numpy()) / len(labels)
  # accuracy = (predictions.max(1)[1] == labels).sum() / len(labels)

  print("Accuracy: {}%".format(accuracy * 100))

  # x_test = images.clone()
  # Step 6: Generate adversarial test examples
  # attack = FastGradientMethod(estimator=model, eps=0.2)
  # x_test_adv = attack.generate(x=x_test)

  # Step 7: Evaluate the ART classifier on adversarial test examples

  # predictions = model.predict(x_test_adv)
  # accuracy = np.sum(np.argmax(predictions, axis=1) == labels.numpy()) / len(labels)
  # print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
  # break

  res0 = clever_t(classifier, images[0].numpy(), target_class=2, nb_batches=10, batch_size=5, radius=R_Linf, norm=np.inf, pool_factor=3)
  print("Targeted CLEVER score:",res0)
  res1 = clever_u(classifier, images[0].numpy(), nb_batches=10, batch_size=5, radius=R_Linf, norm=np.inf, pool_factor=3)
  print("Untargeted CLEVER score:",res1)
  break


# # classifier = model
# for images, labels in test_loader:
#     # images = images.to(device)
#     # labels = labels.to(device)
#     print(images.shape)
#     # y = np.array(images)
#     # print(y.shape)
#     a = art.metrics.clever_u(model, images, nb_batches=2, batch_size=1, radius=0.3, norm=np.inf, verbose = True)
#     break

In [ ]:
from art.estimators.classification.pytorch import PyTorchClassifier
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
classifier = PyTorchClassifier(
  model=model,
  clip_values=(min_pixel_value, max_pixel_value),
  loss=None,
  optimizer=None,
  input_shape=(1, 32, 32),
  nb_classes=10,
)
res1 = clever_u(classifier, images[0].numpy(), nb_batches=10, batch_size=5, radius=R_Linf, norm=np.inf, pool_factor=3)

In [ ]:
"""
The script demonstrates a simple example of using ART with PyTorch. The example train a small model on the MNIST dataset
and creates adversarial examples using the Fast Gradient Sign Method. Here we use the ART classifier to train the model,
it would also be possible to provide a pretrained model to the ART classifier.
The parameters are chosen for reduced computational requirements of the script and not optimised for accuracy.
"""
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist


# Step 0: Define the neural network model, return logits instead of activation in forward method


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
        self.fc_2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 10)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

# Step 2: Create the model

xmodel = Net()

# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=xmodel,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)

# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import numpy as np
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

from art.estimators.classification.keras import KerasClassifier
from art.estimators.classification.pytorch import PyTorchClassifier
from art.estimators.classification.tensorflow import TensorFlowClassifier
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
from art.utils import load_mnist

from tests.utils import master_seed

tf.compat.v1.disable_eager_execution()

batch_size, nb_train, nb_test = 100, 1000, 10
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()
x_train, y_train = x_train[:nb_train], y_train[:nb_train]
x_test, y_test = x_test[:nb_test], y_test[:nb_test]
tfmodel = Sequential()
tfmodel.add(Conv2D(4, kernel_size=(5, 5), activation="relu", input_shape=(28, 28, 1)))
tfmodel.add(MaxPooling2D(pool_size=(2, 2)))
tfmodel.add(Flatten())
tfmodel.add(Dense(10, activation="softmax"))

tfmodel.compile(
loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=["accuracy"]
)

# Get the classifier
krc = KerasClassifier(model=tfmodel, clip_values=(0, 1), use_logits=False)

# Get the classifier
krc.fit(x_train, y_train, batch_size=batch_size, nb_epochs=1, verbose=0)
R_L1 = 40
R_L2 = 2

# Test targeted clever
res0 = clever_t(krc, x_test[-1], 2, 10, 5, R_L1, norm=1, pool_factor=3)
print("Targeted CLEVER score:",res0)
res1 = clever_u(krc, x_test[-1], 10, 5, R_L1, norm=1, pool_factor=3)
print("Untargeted CLEVER score:",res1)

In [ ]:
x_test[-1].dtype, type(x_test[-1])